# Importing Data and Libraries
---

In [94]:
import os
from catboost import CatBoostClassifier
from sklearn.preprocessing import OneHotEncoder
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns

  adding: kaggle/input/datathon24playground/datathon-2024-playground/ (stored 0%)
  adding: kaggle/input/datathon24playground/datathon-2024-playground/sample_submission.csv (deflated 65%)
  adding: kaggle/input/datathon24playground/datathon-2024-playground/ip_address_mapping.csv (deflated 77%)
  adding: kaggle/input/datathon24playground/datathon-2024-playground/train.csv (deflated 60%)
  adding: kaggle/input/datathon24playground/datathon-2024-playground/test.csv (deflated 58%)


In [95]:
base_path_dir = ''
try:
    os.listdir('/kaggle')
    base_path_dir = '/kaggle/input/datathon24playground/datathon-2024-playground'
except:
    base_path_dir = './datathon-24-playground'
print("Base path:",base_path_dir)

Base path: /kaggle/input/datathon24playground/datathon-2024-playground


In [38]:
rename_dict = {
    'id_pengguna': 'user_id',
    'waktu_pendaftaran_akun': 'account_registration_time',
    'waktu_pembelian': 'purchase_time',
    'total_harga_pembelian': 'total_purchase_value',
    'id_perangkat': 'device_id',
    'sumber': 'source',
    'browser': 'browser_type',
    'gender': 'gender',
    'umur': 'age',
    'alamat_IP': 'ip_address',
    'fraud': 'fraud',
    'batas_bawah_alamat_IP': 'ip_lower_bound',
    'batas_atas_alamat_IP': 'ip_upper_bound',
    'negara': 'country'
}

In [85]:
train = pd.read_csv(os.path.join(base_path_dir,'train.csv'))
train.rename(columns=rename_dict,inplace=True)

test = pd.read_csv(os.path.join(base_path_dir,'test.csv'))
test.rename(columns=rename_dict,inplace=True)

ip = pd.read_csv(os.path.join(base_path_dir,'ip_address_mapping.csv'))
ip.rename(columns=rename_dict,inplace=True)

# Data Description
---
The dataset consists of three files:

1. **train.csv and test.csv**
- user_id: Unique identifier for each user registered on the e-commerce platform.
- account_registration_time: The time when the user registered on the e-commerce platform.
- purchase_time: The time when the user made a purchase on the e-commerce platform.
- total_purchase_value: The transaction value of the purchase made by the user.
- device_id: Unique identifier of the device used by the user to access the e-commerce platform.
- source: The source or channel through which the user discovered or accessed the e-commerce
- platform (e.g., advertisement, organic search, etc.).
- browser: The type of browser used by the user to access the e-commerce platform.
- gender: The gender of the user registered on the e-commerce platform.
- age: The age of the user registered on the e-commerce platform.
- ip_address: The IP address used by the user's device when accessing the e-commerce platform.
- fraud: Classification of the transaction as fraudulent or not (Not available in test.csv).

2. **ip_address_mapping.csv**
- ip_lower_bound: The lower bound of the IP address range used to determine the user's geographical location.
- ip_upper_bound: The upper bound of the IP address range used to determine the user's geographical location.
- country: The country of origin of the user based on the IP address used to access the e-commerce platform.

## Train

In [40]:
train.head(5)

,user_id,account_registration_time,purchase_time,total_purchase_value,device_id,source,browser_type,gender,age,ip_address,fraud
0,286873,6/7/2015 12:58,9/27/2015 16:32,14,DNTIEECGGDXHF,Iklan,Opera,F,38,2.417207e+08,0
1,244392,5/1/2015 15:29,8/27/2015 14:21,11,MECRFNJGDPGOB,SEO,Chrome,M,36,4.172794e+09,0
2,20720,3/29/2015 18:04,6/13/2015 6:50,52,WLTSYVLLASJOZ,SEO,Internet Explorer,M,26,1.113433e+09,0
3,198755,1/29/2015 0:49,2/17/2015 21:49,13,TWGNWOWURZFMY,SEO,Safari,F,38,1.339309e+09,0
4,64996,1/17/2015 19:30,3/1/2015 9:17,23,XMQENRCEILYAJ,Langsung,Safari,F,19,4.602382e+08,0


In [103]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75556 entries, 0 to 75555
Data columns (total 11 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   user_id                    75556 non-null  int64  
 1   account_registration_time  75556 non-null  object 
 2   purchase_time              75556 non-null  object 
 3   total_purchase_value       75556 non-null  int64  
 4   device_id                  75556 non-null  object 
 5   source                     75556 non-null  object 
 6   browser_type               75556 non-null  object 
 7   gender                     75556 non-null  object 
 8   age                        75556 non-null  int64  
 9   ip_address                 75556 non-null  float64
 10  fraud                      75556 non-null  int64  
dtypes: float64(1), int64(4), object(6)
memory usage: 6.3+ MB


In [102]:
for col in train.select_dtypes('object'):
    print(f'DESCRIBING: {col}')
    print(train[col].unique())
    print(train[col].nunique())
    print('------------------')

DESCRIBING: account_registration_time
['6/7/2015 12:58' '5/1/2015 15:29' '3/29/2015 18:04' ... '2/18/2015 13:55'
 '4/15/2015 1:21' '1/5/2015 21:56']
70669
------------------
DESCRIBING: purchase_time
['9/27/2015 16:32' '8/27/2015 14:21' '6/13/2015 6:50' ... '5/29/2015 2:41'
 '7/29/2015 7:37' '5/21/2015 5:43']
66979
------------------
DESCRIBING: device_id
['DNTIEECGGDXHF' 'MECRFNJGDPGOB' 'WLTSYVLLASJOZ' ... 'TJYOHNTNXAMYU'
 'VGMIKGWQYPRPI' 'FAYIRBHTCRTHC']
71299
------------------
DESCRIBING: source
['Iklan' 'SEO' 'Langsung']
3
------------------
DESCRIBING: browser_type
['Opera' 'Chrome' 'Internet Explorer' 'Safari' 'Firefox']
5
------------------
DESCRIBING: gender
['F' 'M']
2
------------------


In [22]:
train.describe()

,id_pengguna,total_harga_pembelian,umur,alamat_IP,fraud
count,75556.000000,75556.000000,75556.000000,7.555600e+04,75556.000000
mean,200306.194994,36.862407,33.124649,2.152027e+09,0.079755
std,115197.966115,18.302404,8.625183,1.247912e+09,0.270916
min,4.000000,9.000000,18.000000,5.209350e+04,0.000000
25%,101157.250000,22.000000,27.000000,1.086566e+09,0.000000
50%,200399.500000,34.000000,33.000000,2.151286e+09,0.000000
75%,299563.250000,49.000000,39.000000,3.243383e+09,0.000000
max,399995.000000,154.000000,73.000000,4.294822e+09,1.000000


## Test

In [41]:
test.head(5)

,user_id,account_registration_time,purchase_time,total_purchase_value,device_id,source,browser_type,gender,age,ip_address
0,28006,2015-06-26 13:27:55,2015-07-13 02:46:13,48,IVHYUSXPDUJUD,SEO,Chrome,M,31,4.268419e+09
1,245189,2015-06-23 15:48:00,2015-09-16 07:26:25,46,UJURCZHQSNBQA,SEO,Firefox,M,34,4.277726e+09
2,58345,2015-06-18 04:14:35,2015-09-21 22:24:59,30,PVWDQUCMMBJXJ,Iklan,Firefox,F,20,1.235071e+09
3,218574,2015-05-19 14:55:18,2015-08-21 00:44:36,29,PMSPHBOFWBTRC,Langsung,Safari,M,25,2.496396e+09
4,49779,2015-04-02 23:35:29,2015-07-08 06:24:47,19,NFGHLAQQPTLVY,Iklan,Chrome,M,38,2.701254e+09


In [101]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75556 entries, 0 to 75555
Data columns (total 10 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   user_id                    75556 non-null  int64  
 1   account_registration_time  75556 non-null  object 
 2   purchase_time              75556 non-null  object 
 3   total_purchase_value       75556 non-null  int64  
 4   device_id                  75556 non-null  object 
 5   source                     75556 non-null  object 
 6   browser_type               75556 non-null  object 
 7   gender                     75556 non-null  object 
 8   age                        75556 non-null  int64  
 9   ip_address                 75556 non-null  float64
dtypes: float64(1), int64(3), object(6)
memory usage: 5.8+ MB


In [100]:
for col in test.select_dtypes('object'):
    print(f'DESCRIBING: {col}')
    print(test[col].unique())
    print(test[col].nunique())
    print('------------------')

DESCRIBING: account_registration_time
['2015-06-26 13:27:55' '2015-06-23 15:48:00' '2015-06-18 04:14:35' ...
 '2015-02-18 19:35:42' '2015-06-11 11:25:34' '2015-02-04 03:27:10']
75556
------------------
DESCRIBING: purchase_time
['2015-07-13 02:46:13' '2015-09-16 07:26:25' '2015-09-21 22:24:59' ...
 '2015-06-13 11:32:14' '2015-08-26 16:56:16' '2015-04-21 18:08:33']
75466
------------------
DESCRIBING: device_id
['IVHYUSXPDUJUD' 'UJURCZHQSNBQA' 'PVWDQUCMMBJXJ' ... 'VTNESIHQEDBVT'
 'AFAPNMPPJEHGS' 'EGNGWAPEJUMPO']
70126
------------------
DESCRIBING: source
['SEO' 'Iklan' 'Langsung']
3
------------------
DESCRIBING: browser_type
['Chrome' 'Firefox' 'Safari' 'Internet Explorer' 'Opera']
5
------------------
DESCRIBING: gender
['M' 'F']
2
------------------


In [99]:
test.describe()

,user_id,total_purchase_value,age,ip_address
count,75556.000000,75556.000000,75556.000000,7.555600e+04
mean,200035.886945,37.008338,33.156758,2.152263e+09
std,115540.954156,18.342928,8.610305,1.249090e+09
min,2.000000,9.000000,18.000000,1.058185e+05
25%,100116.750000,22.000000,27.000000,1.085203e+09
50%,199599.500000,35.000000,33.000000,2.159574e+09
75%,300567.250000,49.000000,39.000000,3.243134e+09
max,400000.000000,140.000000,76.000000,4.294850e+09


## IP Address Mapping

In [98]:
ip.head(5)

,ip_lower_bound,ip_upper_bound,country
0,16777216,16777471,Australia
1,16777472,16777727,China
2,16777728,16778239,China
3,16778240,16779263,Australia
4,16779264,16781311,China


In [97]:
ip.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 138846 entries, 0 to 138845
Data columns (total 3 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   ip_lower_bound  138846 non-null  int64 
 1   ip_upper_bound  138846 non-null  int64 
 2   country         138846 non-null  object
dtypes: int64(2), object(1)
memory usage: 3.2+ MB


In [96]:
ip.describe()

,ip_lower_bound,ip_upper_bound
count,1.388460e+05,1.388460e+05
mean,2.724532e+09,2.724557e+09
std,8.975215e+08,8.974979e+08
min,1.677722e+07,1.677747e+07
25%,1.919930e+09,1.920008e+09
50%,3.230887e+09,3.230888e+09
75%,3.350465e+09,3.350466e+09
max,3.758096e+09,3.758096e+09


In [33]:
print(ip['negara'].nunique())
print(ip['negara'].unique())

235
['Australia' 'China' 'Japan' 'Thailand' 'India' 'Malaysia'
 'Korea Republic of' 'Hong Kong' 'Taiwan; Republic of China (ROC)'
 'Philippines' 'Viet Nam' 'France' 'European Union' 'United Kingdom'
 'Italy' 'United Arab Emirates' 'Israel' 'Ukraine' 'Russian Federation'
 'Sweden' 'Kazakhstan' 'Portugal' 'Greece' 'Saudi Arabia' 'Denmark'
 'Spain' 'Iran' 'Norway' 'Germany' 'United States' 'Syrian Arab Republic'
 'Cyprus' 'Czech Republic' 'Switzerland' 'Iraq' 'Netherlands' 'Turkey'
 'Romania' 'Lebanon' 'Hungary' 'Georgia' 'Azerbaijan' 'Austria'
 'Palestina' 'Lithuania' 'Oman' 'Serbia' 'Finland' 'Belgium' 'Bulgaria'
 'Slovenia' 'Moldova Republic of' 'Macedonia' 'Estonia' 'Liechtenstein'
 'Croatia' 'Poland' 'Bosnia and Herzegowina' 'Latvia' 'Jordan'
 'Kyrgyzstan' 'Ireland' 'Libyan Arab Jamahiriya' 'Armenia' 'Yemen'
 'Belarus' 'Gibraltar' 'Luxembourg' 'Slovakia (SLOVAK Republic)' 'Malta'
 'New Zealand' 'Singapore' 'Indonesia' 'Nepal' 'Papua New Guinea'
 'Pakistan' 'Canada' 'Barbados' 'Puerto

# Data Preprocessing
---
Necessary steps:
- Changing date object columns to Pandas DateTime
- Object columns treatment (Drop/Encode)
- Merging train/test with IP Address mapping

In [86]:
# Define date columns
date_cols = [col for col in train.select_dtypes('object') if 'time' in col]
date_cols

def dateTreatment(df_in):
    df = df_in.copy()
    for col in date_cols:
        df[col] = pd.to_datetime(df[col]) # Converting object to DateTime
    return df

train_ready = dateTreatment(train)
test_ready = dateTreatment(test)
train_ready.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75556 entries, 0 to 75555
Data columns (total 11 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   user_id                    75556 non-null  int64         
 1   account_registration_time  75556 non-null  datetime64[ns]
 2   purchase_time              75556 non-null  datetime64[ns]
 3   total_purchase_value       75556 non-null  int64         
 4   device_id                  75556 non-null  object        
 5   source                     75556 non-null  object        
 6   browser_type               75556 non-null  object        
 7   gender                     75556 non-null  object        
 8   age                        75556 non-null  int64         
 9   ip_address                 75556 non-null  float64       
 10  fraud                      75556 non-null  int64         
dtypes: datetime64[ns](2), float64(1), int64(4), object(4)
memory usage:

In [87]:
# Encoding important categorical features
low_cardinality = ['source', 'browser_type','gender']

def encodeFeatures(df_in):
    df = df_in.copy()
    encoder = OneHotEncoder(sparse_output=False)
    one_hot_encoded = encoder.fit_transform(df[low_cardinality])
    one_hot_df = pd.DataFrame(one_hot_encoded, columns=encoder.get_feature_names_out(low_cardinality))
    df = pd.concat([df, one_hot_df], axis=1)
    df = df.drop(low_cardinality, axis=1)
    return df

train_ready = encodeFeatures(train_ready)
test_ready = encodeFeatures(test_ready)
train_ready.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75556 entries, 0 to 75555
Data columns (total 18 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   user_id                         75556 non-null  int64         
 1   account_registration_time       75556 non-null  datetime64[ns]
 2   purchase_time                   75556 non-null  datetime64[ns]
 3   total_purchase_value            75556 non-null  int64         
 4   device_id                       75556 non-null  object        
 5   age                             75556 non-null  int64         
 6   ip_address                      75556 non-null  float64       
 7   fraud                           75556 non-null  int64         
 8   source_Iklan                    75556 non-null  float64       
 9   source_Langsung                 75556 non-null  float64       
 10  source_SEO                      75556 non-null  float64       
 11  br

In [74]:
ip.columns

Index(['ip_lower_bound', 'ip_upper_bound', 'country'], dtype='object')

In [ ]:
# Merging IP address dataset
def find_country(ip_address):
    country = ip[(ip['ip_lower_bound'] <= ip_address) & (ip['ip_upper_bound'] >= ip_address)]['country']
    if not country.empty:
        return country.values[0]
    else:
        return 'Unknown'

train_ready['country'] = train_ready['ip_address'].apply(find_country)
test_ready['country'] = test_ready['ip_address'].apply(find_country)

In [91]:
train_ready.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75556 entries, 0 to 75555
Data columns (total 19 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   user_id                         75556 non-null  int64         
 1   account_registration_time       75556 non-null  datetime64[ns]
 2   purchase_time                   75556 non-null  datetime64[ns]
 3   total_purchase_value            75556 non-null  int64         
 4   device_id                       75556 non-null  object        
 5   age                             75556 non-null  int64         
 6   ip_address                      75556 non-null  float64       
 7   fraud                           75556 non-null  int64         
 8   source_Iklan                    75556 non-null  float64       
 9   source_Langsung                 75556 non-null  float64       
 10  source_SEO                      75556 non-null  float64       
 11  br

The remaining object columns have the potential to be explored through **Feature Engineering**!





# Exploratory Data Analysis
---

In [ ]:
# isi sendiri

# Feature Engineering
---

In [ ]:
# isi sendiri

# Modelling
---

In [ ]:
# assignment selanjutnya